### Installations

In [1]:
!pip install -q albumentations

### Importations

In [2]:
# PyTorch:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchvision import transforms
from torch.cuda.amp import autocast, GradScaler

# Traitement image:
import numpy as np
import pandas as pd
from scipy.ndimage import distance_transform_edt
from skimage.measure import label
from PIL import Image
import cv2
import nibabel as nib
from collections import defaultdict

# Visualisation:

import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# Files Management:
import os
import zipfile
import wandb
from google.colab import drive

# Device:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
# print(torch.__version__)

In [3]:
drive.mount('/content/drive')
zip_path = "/content/drive/MyDrive/dataset.zip"  # chemin zip dans Drive
extract_path = "/content/dataset"                # où décompresser dans Colab

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Décompression terminée")

Mounted at /content/drive
Décompression terminée


### Création des Datasets:

In [4]:
class NiftiSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=False, class_map=None):
        self.image_paths = []
        self.mask_paths = []
        self.transform = transform
        self.class_map = class_map

        image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.nii.gz')])
        mask_files = set(os.listdir(mask_dir))

        for img_file in image_files:
            index = os.path.splitext(os.path.splitext(img_file)[0])[0]  # remove .nii.gz
            mask_file = f"{index}.nii.gz"
            if mask_file in mask_files:
                self.image_paths.append(os.path.join(image_dir, img_file))
                self.mask_paths.append(os.path.join(mask_dir, mask_file))

        print(f"Matched {len(self.image_paths)} image/mask pairs.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = nib.load(self.image_paths[idx]).get_fdata()
        mask = nib.load(self.mask_paths[idx]).get_fdata()

        z = image.shape[-1] // 2
        image = image[:, :, z]
        mask = mask[:, :, z]

        if self.class_map:
            mask = remap_mask(mask, self.class_map)

        if self.transform:
            augmented = self.transform(image=image.astype(np.float32), mask=mask.astype(np.uint8))
            image = augmented["image"]                     # (1, H, W) torch.Tensor
            mask = augmented["mask"].long()                # (H, W) torch.LongTensor
        else:
            image = torch.tensor(image / image.max(), dtype=torch.float32).unsqueeze(0)
            mask = torch.tensor(mask, dtype=torch.long)

        return image, mask



def generate_image_batch(data_batch: list) -> tuple:
    """
    Assemble a batch of image/mask pairs into batched tensors.

    Args:
        data_batch (list): List of (image, mask) tuples.
            - image: Tensor of shape (C, H, W)
            - mask : Tensor of shape (H, W)

    Returns:
        image_batch (Tensor): Batched images of shape (B, C, H, W)
        mask_batch  (Tensor): Batched masks of shape (B, H, W)
    """
    images, masks = zip(*data_batch)
    image_batch = torch.stack(images)  # Shape: (B, C, H, W)
    mask_batch = torch.stack(masks)    # Shape: (B, H, W)
    return image_batch, mask_batch

def remap_mask(mask: np.ndarray, mapping: dict) -> np.ndarray:
    """
    Remappe les valeurs des masques selon un dictionnaire de correspondance.

    Args:
        mask (np.ndarray): masque original
        mapping (dict): ex. {0:0, 2:1, 3:2, 4:3, 5:4}

    Returns:
        np.ndarray: masque remappé avec classes consécutives
    """
    remapped = np.zeros_like(mask, dtype=np.uint8)
    for old_val, new_val in mapping.items():
        remapped[mask == old_val] = new_val
    return remapped

def verify_dataset(name, dataset):
    print(f"\nVérification du dataset : {name}")
    print(f"Nombre d'exemples : {len(dataset)}")

    # Charger un exemple
    try:
        img, msk = dataset[0]
        print(f"Image shape : {img.shape} | dtype : {img.dtype} | min/max : {img.min().item():.2f}/{img.max().item():.2f}")
        print(f"Masque shape : {msk.shape} | dtype : {msk.dtype} | unique values : {torch.unique(msk)}")

        # Affichage
        plt.figure(figsize=(10, 4))
        plt.suptitle(name)
        plt.subplot(1, 2, 1)
        plt.title("Image")
        plt.imshow(img.squeeze(), cmap="gray")
        plt.subplot(1, 2, 2)
        plt.title("Masque")
        plt.imshow(msk, cmap="jet")
        plt.show()

    except Exception as e:
        print(f"Erreur lors de la lecture d'un échantillon : {e}")

In [5]:
print("Contenu du dossier extrait :", os.listdir(extract_path))

base_path_nii = "/content/dataset/dataset/mbh/nii"

# TRAIN
train_image_dir = os.path.join(base_path_nii, "train", "img")
train_mask_dir  = os.path.join(base_path_nii, "train", "seg")

# VAL
val_image_dir = os.path.join(base_path_nii, "val", "img")
val_mask_dir  = os.path.join(base_path_nii, "val", "seg")

# TEST
test_image_dir = os.path.join(base_path_nii, "test", "img")
test_mask_dir  = os.path.join(base_path_nii, "test", "seg")

train_dataset = NiftiSegmentationDataset(train_image_dir, train_mask_dir)

# Mapping des classes : [0, 2, 3, 4, 5] → [0, 1, 2, 3, 4]
CLASS_MAPPING = {0: 0, 2: 1, 3: 2, 4: 3, 5: 4}

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Affine(rotate=(-25, 25), shear=(-10, 10), p=0.5),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.Normalize(mean=0.0, std=1.0, max_pixel_value=1.0),
    A.pytorch.ToTensorV2()
])

train_dataset = NiftiSegmentationDataset(
                                        image_dir=train_image_dir,
                                        mask_dir=train_mask_dir,
                                        transform=train_transform,
                                        class_map=CLASS_MAPPING
                                        )

# for i in range(5):
#     _, mask = train_dataset[i]
#     unique_vals = torch.unique(mask)
#     print(f"Masque {i} — valeurs uniques :", unique_vals)

#     if unique_vals.numel() <= 1:
#         print("⚠️  Masque pauvre en classes. Potentiellement inutile pour l'entraînement.")


val_transform = A.Compose([
    A.Normalize(mean=0.0, std=1.0, max_pixel_value=1.0),
    A.pytorch.ToTensorV2()
])

val_dataset = NiftiSegmentationDataset(
                                      image_dir=val_image_dir,
                                      mask_dir=val_mask_dir,
                                      transform=val_transform,
                                      class_map=CLASS_MAPPING
                                      )





test_dataset  = NiftiSegmentationDataset(test_image_dir, test_mask_dir)

# print("\nContenu de train/img :")
# print(os.listdir("/content/dataset/dataset/mbh/nii/train/img")[0])
# print("\nContenu de train/seg :")
# print(os.listdir("/content/dataset/dataset/mbh/nii/train/seg")[0])


# # Appels pour les 3 datasets
# verify_dataset("TRAIN", train_dataset)
# verify_dataset("VAL", val_dataset)
# verify_dataset("TEST", test_dataset)

Contenu du dossier extrait : ['dataset']
Matched 155 image/mask pairs.
Matched 155 image/mask pairs.
Matched 17 image/mask pairs.
Matched 50 image/mask pairs.


#### Version avec tif dataset:

In [6]:
# ### Test avec la dataset TIF ####

# class CellSegmentationDataset(Dataset):
#     def __init__(self, image_dir, mask_dir, transform=None):
#         self.image_paths = []
#         self.mask_paths = []
#         self.transform = transform

#         image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.tif')])
#         mask_files = set(os.listdir(mask_dir))  # pour recherche rapide

#         for img_file in image_files:
#             # Exemple : t005.tif → index = '005'
#             index = os.path.splitext(img_file)[0][1:]  # enlève 't' et '.tif'
#             mask_file = f"man_seg{index}.tif"
#             if mask_file in mask_files:
#                 self.image_paths.append(os.path.join(image_dir, img_file))
#                 self.mask_paths.append(os.path.join(mask_dir, mask_file))

#         print(f"Matched {len(self.image_paths)} image/mask pairs.")

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image = Image.open(self.image_paths[idx]).convert("L")  # grayscale
#         mask = Image.open(self.mask_paths[idx]).convert("L")    # grayscale mask

#         if self.transform:
#             image = self.transform(image)
#             mask = self.transform(mask)
#         else:
#             image = transforms.ToTensor()(image)
#             mask = Image.open(self.mask_paths[idx])
#             mask = np.array(mask, dtype=np.uint8)
#             mask = torch.from_numpy(mask).long()

#         # mask: (1, H, W) → (H, W) avec classes entières
#         mask = mask.squeeze().long()

#         return image, mask



# !wget https://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip -O DIC-C2DH-HeLa.zip
# !unzip -oq DIC-C2DH-HeLa.zip -d DIC-C2DH-HeLa

# base_path = 'DIC-C2DH-HeLa'

# image_dir = "DIC-C2DH-HeLa/DIC-C2DH-HeLa/01"
# mask_dir = "DIC-C2DH-HeLa/DIC-C2DH-HeLa/01_GT/SEG"

# dataset = CellSegmentationDataset(image_dir, mask_dir)

# img, msk = dataset[0]


# plt.figure(figsize=(10, 4))
# plt.subplot(1, 2, 1)
# plt.title("Image")
# plt.imshow(img.squeeze(), cmap="gray")

# plt.subplot(1, 2, 2)
# plt.title("Mask")
# plt.imshow(msk, cmap="jet")
# plt.show()

## Classe U-Net

In [7]:
class DoubleConv(nn.Module):
    """Applies two convolutions 3x3 + ReLU"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = self.down_block(64, 128)
        self.down2 = self.down_block(128, 256)
        self.down3 = self.down_block(256, 512)
        self.down4 = self.down_block(512, 1024)

        self.up1 = self.up_block(1024, 512)
        self.up2 = self.up_block(512, 256)
        self.up3 = self.up_block(256, 128)
        self.up4 = self.up_block(128, 64)

        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def down_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def up_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.crop_and_concat(self.up1[0](x5), x4)
        x = self.up1[1](x)

        x = self.crop_and_concat(self.up2[0](x), x3)
        x = self.up2[1](x)

        x = self.crop_and_concat(self.up3[0](x), x2)
        x = self.up3[1](x)

        x = self.crop_and_concat(self.up4[0](x), x1)
        x = self.up4[1](x)

        logits = self.outc(x)
        return logits

    def crop_and_concat(self, upsampled, bypass):
        _, _, H, W = upsampled.shape
        bypass_cropped = self.center_crop(bypass, H, W)
        return torch.cat([bypass_cropped, upsampled], dim=1)

    def center_crop(self, tensor, target_h, target_w):
        _, _, h, w = tensor.shape
        start_x = (w - target_w) // 2
        start_y = (h - target_h) // 2
        return tensor[:, :, start_y:start_y + target_h, start_x:start_x + target_w]

def count_unique_labels(dataset):
    unique_labels = set()
    for i in range(len(dataset)):
        _, mask = dataset[i]
        unique_labels.update(torch.unique(mask).tolist())
    return sorted(unique_labels)

### Pondération de classe:

In [8]:
def compute_weight_map(mask, w0=20, sigma=3):
    """
    Compute pixel-wise weight map for segmentation training.

    Args:
        mask (np.array): Labeled 2D mask.
        w0 (float): Weight for separation term.
        sigma (float): Std-dev for border emphasis.

    Returns:
        np.array: weight map w(x)
    """
    labeled_mask = label(mask)
    num_objects = np.max(labeled_mask)

    if num_objects < 2:
        return np.ones_like(mask, dtype=np.float32)

    # Distances aux autres objets
    dists = np.zeros((num_objects, *mask.shape), dtype=np.float32)
    for i in range(1, num_objects + 1):
        dists[i - 1] = distance_transform_edt(labeled_mask != i)

    d1 = np.min(dists, axis=0)
    dists[dists == d1] = np.inf
    d2 = np.min(dists, axis=0)
    separation = w0 * np.exp(-((d1 + d2) ** 2) / (2 * sigma ** 2))

    # Pondération par fréquence de classe
    class_weights = np.ones_like(mask, dtype=np.float32)
    labels, counts = np.unique(mask, return_counts=True)
    total = np.sum(counts)
    freqs = {val: count / total for val, count in zip(labels, counts)}

    for val in labels:
        # if val == 0:
        #     continue
        class_weights[mask == val] = 1.0 / (freqs[val] + 1e-6)

    # Normaliser pour stabiliser:
    class_weights /= np.mean(class_weights)

    return class_weights + separation


def compute_weight_batch(batch_masks):
    """
    Compute the weight map for each mask in a batch.
    Returns a tensor of shape (B, H, W)
    """
    weights = []
    for mask in batch_masks:
        mask_np = mask.cpu().numpy()
        mask_np = (mask_np > 0).astype(np.uint8)  # binarize the mask
        w = compute_weight_map(mask_np)
        weights.append(torch.tensor(w, dtype=torch.float32))
    return torch.stack(weights).to(device)


def weighted_cross_entropy_loss(logits, targets, weight_map):
    """
    Computes the pixel-wise weighted cross-entropy loss as defined in Eq. (1) of the U-Net paper.

    Parameters:
        logits (Tensor): Network output, shape (B, C, H, W), unnormalized scores for each class.
        targets (Tensor): Ground truth segmentation, shape (B, H, W), with class indices.
        weight_map (Tensor): Weight map w(x), shape (B, H, W), computed from the ground truth mask.

    Returns:
        Tensor: Scalar loss value.
    """
    assert logits.device == targets.device == weight_map.device
    assert logits.dtype in [torch.float32, torch.float16, torch.bfloat16]
    assert weight_map.dtype in [torch.float32, torch.float16, torch.bfloat16]

    B, C, H, W = logits.shape

    # Compute log probabilities
    log_probs = F.log_softmax(logits, dim=1)

    # Select log-probabilities of the correct class at each pixel
    log_probs_target = log_probs.gather(1, targets.unsqueeze(1)).squeeze(1)

    # Apply pixel-wise weights
    loss = -weight_map * log_probs_target
    return loss.mean()

### Fonctions d'évaluations:

In [9]:
def dice_loss(logits: torch.Tensor, targets: torch.Tensor, smooth: float = 1.0) -> torch.Tensor:
    """
    Compute multi-class Dice Loss between logits and ground truth labels.

    Args:
        logits (Tensor): Shape (B, C, H, W) — raw output from the model (before softmax)
        targets (Tensor): Shape (B, H, W) — ground truth class indices
        smooth (float): smoothing constant to avoid division by zero

    Returns:
        Tensor: Dice loss value
    """
    num_classes = logits.shape[1]
    probs = F.softmax(logits, dim=1)                    # (B, C, H, W)
    targets_one_hot = F.one_hot(targets, num_classes)   # (B, H, W, C)
    targets_one_hot = targets_one_hot.permute(0, 3, 1, 2).float()  # (B, C, H, W)

    intersection = (probs * targets_one_hot).sum(dim=(0, 2, 3))
    union = probs.sum(dim=(0, 2, 3)) + targets_one_hot.sum(dim=(0, 2, 3))

    dice = (2. * intersection + smooth) / (union + smooth)
    loss = 1.0 - dice.mean()

    return loss


def topk_pixel_accuracy(logits: torch.Tensor, targets: torch.Tensor, k: int) -> float:
    """
    Compute pixel-wise top-k accuracy for segmentation.

    Args:
        logits (Tensor): shape (B, C, H, W)
        targets (Tensor): shape (B, H, W)
        k (int): top-k value

    Returns:
        float: top-k pixel accuracy
    """
    B, C, H, W = logits.shape
    targets_flat = targets.view(B, -1)
    logits_flat = logits.view(B, C, -1).transpose(1, 2)  # (B, H*W, C)

    topk = logits_flat.topk(k, dim=-1).indices           # (B, H*W, k)
    targets_exp = targets_flat.unsqueeze(-1).expand_as(topk)  # (B, H*W, k)

    correct = (topk == targets_exp).any(dim=-1).float()
    return correct.mean().item()


def eval_model(model: nn.Module, dataloader: DataLoader, config: dict, topk_list=[1, 3, 5]) -> dict:
    """
    Evaluate U-Net on a dataset and compute multiple metrics.

    Args:
        model (nn.Module): Trained segmentation model.
        dataloader (DataLoader): Validation or test loader.
        config (dict): Must contain 'device', 'loss', and optionally 'weight_map_fn'.
        topk_list (list): List of top-k accuracies to compute.

    Returns:
        dict: Averaged metrics over the dataset: loss, accuracy, mean IoU, top-k.
    """
    device = config["device"]
    loss_fn = config["loss"]
    weight_map_fn = config.get("weight_map_fn", None)
    logs = defaultdict(list)

    model.eval()
    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            ## Fonction Dice_loss ou Cross_Entropy_Loss seulement
            # if weight_map_fn:
            #     weight_map = weight_map_fn(masks).to(device)
            #     loss = loss_fn(outputs, masks, weight_map)
            # else:
            #     loss = loss_fn(outputs, masks)

            # Mix Dice_Loss + Cross_Entropy

            if weight_map_fn:
                weight_map = weight_map_fn(masks).to(device)
                ce_loss = loss_fn(outputs, masks, weight_map)
            else:
                ce_loss = loss_fn(outputs, masks)

            dice = dice_loss(outputs, masks)
            loss = 0.3 * ce_loss + 0.7 * dice
            logs["loss"].append(loss.item())

            # Pixel accuracy
            correct = (preds == masks).float().mean().item()
            logs["accuracy"].append(correct)

            # Per-class IoU (excluding background = 0)
            ious = []
            num_classes = outputs.shape[1]
            for cls in range(1, num_classes):  # ignore background
                pred_cls = (preds == cls)
                mask_cls = (masks == cls)
                intersection = torch.logical_and(pred_cls, mask_cls).sum().float()
                union = torch.logical_or(pred_cls, mask_cls).sum().float()
                if union > 0:
                    iou = intersection / (union + 1e-6)
                    ious.append(iou.item())

            mean_iou = np.mean(ious) if ious else 0.0
            logs["IoU"].append(mean_iou)

            # Top-k pixel accuracy
            for k in topk_list:
                acc_k = topk_pixel_accuracy(outputs, masks, k)
                logs[f"top-{k}"].append(acc_k)

    return {k: np.mean(v) for k, v in logs.items()}

def loss_batch_unet(
                    model: nn.Module,
                    images: torch.Tensor,
                    masks: torch.Tensor,
                    config: dict
                    ) -> dict:
    """
    Compute loss and metrics for a U-Net batch (segmentation).

    Args:
        model (nn.Module): U-Net model
        images (Tensor): shape (B, C, H, W)
        masks (Tensor): shape (B, H, W) with integer class labels
        config (dict): contains:
            - 'device': torch.device
            - 'loss': loss function
            - 'weight_map_fn': function that generates weight maps from masks

    Returns:
        dict: {'loss': ..., 'iou': ..., 'accuracy': ...}
    """
    device = config['device']
    loss_fn = config['loss']
    weight_map_fn = config.get('weight_map_fn', None)

    images = images.to(device)
    masks = masks.to(device)
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(images)  # shape: (B, C, H, W)

        if weight_map_fn:
            weight_map = weight_map_fn(masks)
            loss = loss_fn(outputs, masks, weight_map)
        else:
            loss = loss_fn(outputs, masks)

        # Get predicted classes
        preds = torch.argmax(outputs, dim=1)  # (B, H, W)

        # IoU
        intersection = torch.logical_and(preds == 1, masks == 1).sum(dim=(1, 2)).float()
        union = torch.logical_or(preds == 1, masks == 1).sum(dim=(1, 2)).float()
        iou = (intersection / (union + 1e-6)).mean().item()

        # Pixel accuracy
        correct = (preds == masks).float().mean().item()

    return {'loss': loss.item(), 'iou': iou, 'accuracy': correct}


def show_predictions(model, dataset, device, n=3):
    """
    Visualize `n` predictions from the dataset using the trained model.

    Args:
        model (nn.Module): Trained U-Net model
        dataset (Dataset): PyTorch dataset (ex: val_dataset)
        device (torch.device): "cuda" or "cpu"
        n (int): Number of samples to display
    """
    model.eval()
    n = min(n, len(dataset))  # ne pas dépasser la taille du dataset

    plt.figure(figsize=(12, 4 * n))
    with torch.no_grad():
        for i in range(n):
            image, true_mask = dataset[i]
            image = image.to(device).unsqueeze(0)  # (1, 1, H, W)
            pred = model(image)  # (1, C, H, W)
            pred_mask = torch.argmax(pred.squeeze(), dim=0).cpu()

            plt.subplot(n, 3, i * 3 + 1)
            plt.title("Input Image")
            plt.imshow(image.squeeze().cpu().numpy(), cmap="gray")
            plt.axis("off")


def print_logs(dataset_type: str, logs: dict):
    """
    Nicely format and print log metrics for a given dataset type.

    Args:
        dataset_type (str): "Train", "Val", or "Test"
        logs (dict): Dictionary of metric names and values (floats)
    """
    formatted = [
        f"{name}: {value:.8f}"
        for name, value in logs.items()
    ]
    desc = f"{dataset_type} —\t" + "\t".join(formatted)
    print(desc)


def combined_loss(logits, targets, weight_map=None, alpha=0.5):
    dice = dice_loss(logits, targets)
    if weight_map is not None:
        ce = weighted_cross_entropy_loss(logits, targets, weight_map)
    else:
        ce = F.cross_entropy(logits, targets)
    return alpha * ce + (1 - alpha) * dice

## Training Loop:

In [10]:
def train_and_validate(
                      model,
                      train_loader,
                      val_loader,
                      optimizer,
                      config,
                      n_epochs,
                      log_wandb=True,
                      show_preds=None,
                      dataset_for_display=None,
                      n_logged_images=3,
                      save_path="best_model.pth"
                  ):
    device = config["device"]
    loss_fn = config["loss"]
    weight_map_fn = config.get("weight_map_fn", None)
    best_val_iou = 0.0  # Pour sauvegarde automatique
    num_used_batches = 0
    if log_wandb:
        log_table = wandb.Table(columns=["epoch", "image", "ground_truth", "prediction"])

    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch + 1}/{n_epochs}")
        model.train()
        total_loss = 0.0

        for images, masks in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            if torch.all(masks == 0):
                print(" Batch sans aucune classe foreground — ignoré")
                continue
            num_used_batches += 1

            optimizer.zero_grad()
            images = images.to(device)
            masks = masks.to(device)

            weight_map = weight_map_fn(masks) if weight_map_fn else None
            if weight_map is not None:
                weight_map = weight_map.to(device)
            outputs = model(images)

            assert masks.max() < outputs.shape[1], \
                f"Target max label {masks.max().item()} >= n_classes={outputs.shape[1]}"
            loss = loss_fn(outputs, masks, weight_map) if weight_map is not None else loss_fn(outputs, masks)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / num_used_batches
        print(f"Training Loss: {avg_train_loss:.8f}")

        # Validation avec toutes les métriques
        val_logs = eval_model(model, val_loader, config, topk_list=[1, 3, 5])
        print_logs("Validation", val_logs)

        val_iou = val_logs.get("IoU", 0.0)
        if val_iou > best_val_iou:
            print(f"New best IoU: {val_iou:.8f} (previous: {best_val_iou:.8f}) — saving model.")
            torch.save(model.state_dict(), save_path)
            best_val_iou = val_iou

        if log_wandb and (epoch + 1) % 1 == 0:
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                **{f"val_{k}": v for k, v in val_logs.items()}
            })

        # Visualisation prédictions et logging images
        if show_preds and dataset_for_display is not None and (epoch + 1) % 5 == 0:
            model.eval()
            with torch.no_grad():
                for i in range(min(n_logged_images, len(dataset_for_display))):
                    if i%5 !=0:
                        continue
                    img, mask = dataset_for_display[i]
                    input_tensor = img.unsqueeze(0).to(device)
                    pred = model(input_tensor)
                    pred_mask = torch.argmax(pred.squeeze(), dim=0).cpu()

                    if log_wandb:
                        wandb.log({
                            f"Example/{i}/image": wandb.Image(img.squeeze().cpu().numpy(), caption="Input"),
                            f"Example/{i}/mask": wandb.Image(mask.cpu().numpy(), caption="Ground Truth"),
                            f"Example/{i}/prediction": wandb.Image(pred_mask.numpy(), caption="Prediction")
                        })
                        log_table.add_data(
                            epoch + 1,
                            wandb.Image(img.squeeze().cpu().numpy(), caption="Input"),
                            wandb.Image(mask.cpu().numpy(), caption="Ground Truth"),
                            wandb.Image(pred_mask.numpy(), caption="Prediction")
                        )

    if log_wandb:
        wandb.log({"Predictions Table": log_table})
        print("Final wandb table logged.")


## Evaluation:

In [14]:
batch_sizes = [4, 8, 16]
epoch_values = [10, 25, 50, 100, 200, 300, 600]
learning_rates = [1e-3, 1e-4, 5e-4, 1e-5]
torch.cuda.empty_cache()

# batch_sizes = [16]
# epoch_values = [40]
# learning_rates = [1e-4]

n_classes = 6

for lr in learning_rates:
    print()
    print()
    print()
    print(lr)
    for epochs in epoch_values:
      torch.cuda.empty_cache()
      print(epochs)
      print()
      print()
      print()
      for batch_size in batch_sizes:
        model = UNet(n_channels=1, n_classes=n_classes)
        optimizer = optim.Adam(model.parameters(), lr=lr)

        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            collate_fn=generate_image_batch,
            num_workers= 4,
            pin_memory=True,
            drop_last=True
            )
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=generate_image_batch,
            num_workers= 4,
            pin_memory=True,
            drop_last=True
            )

        config = {
            "architecture": "U-Net",
            "loss": "Weighted Cross Entropy",
            "epochs": epochs,
            "batch_size": batch_size,
            "optimizer": "Adam",
            "lr": lr,
            "n_classes": n_classes,
            "dataset": base_path_nii,
            "device": device,
            "loss_fn": weighted_cross_entropy_loss,
            "train_loader": train_loader,
            "val_loader": val_loader,
            "optimizer_instance": optimizer,
            "weight_map_fn": compute_weight_batch
        }

        # config = {
        #           "architecture": "U-Net",
        #           "loss": "Dice Loss",
        #           "epochs": epochs,
        #           "batch_size": batch_size,
        #           "optimizer": "Adam",
        #           "lr": lr,
        #           "n_classes": n_classes,
        #           "dataset": base_path_nii,
        #           "device": device,
        #           "loss_fn": lambda logits, targets, weight_map=None: dice_loss(logits, targets),
        #           "train_loader": train_loader,
        #           "val_loader": val_loader,
        #           "optimizer_instance": optimizer,
        #           "weight_map_fn": None  # pas avec DiceLoss
        #           }
        model.to(config["device"])

        run_name = f"U-Net_bs{batch_size}_ep{epochs}"

        with wandb.init(
            config=config,
            project=f"INF8225 - Projet U-Net - LR = {lr:.0e}",
            group="U-Net Cell Segmentation",
            name=run_name,
            save_code=True,
        ):
            train_and_validate(
                model=model,
                train_loader=config["train_loader"],
                val_loader=config["val_loader"],
                optimizer=config["optimizer_instance"],
                config={
                    "device": config["device"],
                    "loss": config["loss_fn"],
                    "weight_map_fn": config["weight_map_fn"]
                },
                n_epochs=config["epochs"],
                log_wandb=True,
                show_preds=True,
                dataset_for_display=val_dataset
            )





0.001
10





<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alexlapointe (alexlapointe-polytechnique-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Epoch 1/10


Training Epoch 1: 100%|██████████| 38/38 [00:11<00:00,  3.31it/s]

...

Training Epoch 388: 100%|██████████| 19/19 [00:11<00:00,  1.69it/s]

Training Loss: 0.00132359


Validation —	loss: 0.61774984	accuracy: 0.95906043	IoU: 0.12885832	top-1: 0.95906043	top-3: 0.99142194	top-5: 1.00000000

Epoch 389/600


Training Epoch 389:  32%|███▏      | 6/19 [00:04<00:10,  1.25it/s]
Traceback (most recent call last):
  File "<ipython-input-14-edbc84a60bb2>", line 90, in <cell line: 0>
    train_and_validate(
  File "<ipython-input-10-7063c904e256>", line 37, in train_and_validate
    weight_map = weight_map_fn(masks) if weight_map_fn else None
                 ^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-5e0666eb3855>", line 55, in compute_weight_batch
    w = compute_weight_map(mask_np)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-5e0666eb3855>", line 22, in compute_weight_map
    dists[i - 1] = distance_transform_edt(labeled_mask != i)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/scipy/ndimage/_morphology.py", line 2472, in distance_transform_edt
    input = np.atleast_1d(np.where(input, 1, 0).astype(np.int8))
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 